# BARBERO DURMIENTE

Se ha llevado a cabo la implementación a continuación para dar respuesta al problema del barbero durmiente mediante programación concurrente. Algunos detalles relevantes de la implementación.

La clase barbería tienes una cantidad escueta de atributos y ningún método aparte del constructor. En ella definimos los semáforos que luego utilizamos para ordenar el flujo de las acciones. También guardamos la etiqueta de si el barbero duerme, puesto que he decidido por resultarme más natural no definir una clase barbero ni una clase cliente (aunque estos van a ser los hilos que van a ir trabajando). Lo que hago es definir un hilo barbero que ejecuta la actividadBarbero y 6 hilos cliente que ejecutan la actividad cliente. A ambas actividades les paso el objeto Barberia instanciado inicialmente y que sirve como instrumento de control. Defino un semáforo que sirve como exclusión mutua en todos los hilos, de manera que cada hilo tiene que poder adquirir dicho semáforo para poder proceder con su ejecución (para liberarlo al final). Serviría indistintamente con un cerrojo, pero he encontrado más dinámico el semáforo. El semáforo sillon (que de nuevo podría ser un lock), es redundante en esta implementación, puesto que el acceso al sillón viene controlado por la actividad del Barbero y debería ser redundante, pero controla que no se de paso a un cliente cuando ya hay otro sentado.

Por otro lado, tenemos el semaforosala inicializado a 3 que me sirve para poder dejar esperando fuera a los clientes que llegan cuando la sala está completa, y que solo entran una vez algún cliente de la sala pase a cortarse el pelo.

En el código he incluido algunos time.sleep() para que sirvieran de delay en las actividades de barbero y cliente, y la ejecución fuera más visual.

También he usado el objeto Queue de la librería queue para los clientes que hubiera en la sala, con el objetivo de que el barbero los atienda en orden de llegada.

Por último, he incluido con condición de salida del bucle de la actividad del barbero que terminase cuando hubiera atendido a 6 clientes.

In [1]:
import threading
import time
import random
import queue

class Barberia:
    clientesenSala = queue.Queue()
    clientes_atendidos = []
    def __init__(self):       
        self.mutualex = threading.Semaphore(1) 
        self.sillon = threading.Semaphore(1)
        self.semaforosala = threading.Semaphore(3)
        self.barberodormido = True    

def actividadBarbero(Barberia):
    print("El barbero está durmiendo...\n")
    while len(Barberia.clientes_atendidos) < 6:
        time.sleep(3) # 3 segundos de delay en la actividad del barbero
        Barberia.mutualex.acquire()
        if Barberia.clientesenSala.qsize() == 0:
            print("El barbero sigue durmiendo\n")
            Barberia.mutualex.release()                
        else:
            if Barberia.barberodormido:
                print("El barbero se despierta \n") 
                Barberia.barberodormido = False                
            Barberia.sillon.acquire() 
            print("El barbero da paso a un cliente\n")            
            cliente = Barberia.clientesenSala.get()
            Barberia.semaforosala.release()
            print("El barbero le está cortando el pelo al cliente {}\n".format(cliente))
            time.sleep(random.randint(1,5))
            print("Ha terminado de cortar el pelo al cliente {}\n".format(cliente))
            Barberia.sillon.release()
            Barberia.clientes_atendidos.append(cliente)
            if Barberia.clientesenSala.qsize() == 0:
                print("El barbero se echa a dormir de nuevo\n")
                Barberia.barberodormido = True
            Barberia.mutualex.release()
            
            
            
def actividadCliente(cliente, Barberia):
    time.sleep(random.randint(3,10)) # con esto pretendo que los clientes lleguen un poco más espaciados a la barbería
    print("El cliente {} ha llegado a la barbería\n".format(cliente))
    Barberia.mutualex.acquire()
    if Barberia.clientesenSala.qsize() < 3:
        Barberia.clientesenSala.put(cliente)
        Barberia.semaforosala.acquire()
        print("El cliente {} se sienta en la sala de espera\n".format(cliente))
        Barberia.mutualex.release()
    else:
        print("El cliente {} tiene que esperar fuera a que se libere algún hueco en la sala\n".format(cliente))
        Barberia.mutualex.release()
        Barberia.semaforosala.acquire() # ponemos a este cliente a esperar hasta que se libere alguna silla en la sala de espera
        print("El cliente {} entra en la sala de espera.\n".format(cliente))
        Barberia.clientesenSala.put(cliente)


if __name__=="__main__":
        barberia= Barberia()
        
        barbero = threading.Thread(target=actividadBarbero, args=(barberia,))
        clientes = []
        nombres=['A','B','C','D','E','F']
        for i in range(len(nombres)):            
           clientes.append(threading.Thread(target=actividadCliente, args=(nombres[i], barberia,)))
        
        barbero.start()
        for cliente in clientes:
            cliente.start()
        
        barbero.join()
        for cliente in clientes:
            cliente.join()

El barbero está durmiendo...

El barbero sigue durmiendo

El cliente F ha llegado a la barbería

El cliente F se sienta en la sala de espera

El cliente B ha llegado a la barbería

El cliente B se sienta en la sala de espera

El barbero se despierta 

El barbero da paso a un cliente

El barbero le está cortando el pelo al cliente F

El cliente C ha llegado a la barbería

El cliente E ha llegado a la barbería

El cliente A ha llegado a la barbería
El cliente D ha llegado a la barbería


Ha terminado de cortar el pelo al cliente F

El cliente C se sienta en la sala de espera

El cliente E se sienta en la sala de espera

El cliente D tiene que esperar fuera a que se libere algún hueco en la sala

El cliente A tiene que esperar fuera a que se libere algún hueco en la sala

El barbero da paso a un cliente

El barbero le está cortando el pelo al cliente B

El cliente D entra en la sala de espera.

Ha terminado de cortar el pelo al cliente B

El barbero da paso a un cliente

El barbero le est

Es interesante ver el anterior problema, pero cambiando un poco las condiciones. Abajo vuelvo a poner el código anterior, pero con algunas modificaciones. Estas son: el barbero tarda más en cortar el pelo, los clientes se van cuando al llegar encuentran la sala de espera llena, hay 15 clientes que van a ir visitando la barbería, y la actividad del barbero cesa cuando ha atendido a 5 clientes.

In [ ]:
class Barberia:
    clientesenSala = queue.Queue()
    clientes_atendidos = []
    def __init__(self):       
        self.mutualex = threading.Semaphore(1) 
        self.sillon = threading.Semaphore(1)
        self.semaforosala = threading.Semaphore(3)
        self.barberodormido = True    

def actividadBarbero(Barberia):
    print("El barbero está durmiendo...\n")
    while len(Barberia.clientes_atendidos) < 5:
        time.sleep(2) # 3 segundos de delay en la actividad del barbero
        Barberia.mutualex.acquire()
        if Barberia.clientesenSala.qsize() == 0:
            print("El barbero sigue durmiendo\n")
            Barberia.mutualex.release()                
        else:
            if Barberia.barberodormido:
                print("El barbero se despierta \n") 
                Barberia.barberodormido = False                
            Barberia.sillon.acquire() 
            print("El barbero da paso a un cliente\n")            
            cliente = Barberia.clientesenSala.get()
            Barberia.semaforosala.release()
            print("El barbero le está cortando el pelo al cliente {}\n".format(cliente))
            time.sleep(4)
            print("Ha terminado de cortar el pelo al cliente {}\n".format(cliente))
            Barberia.sillon.release()
            Barberia.clientes_atendidos.append(cliente)
            if Barberia.clientesenSala.qsize() == 0:
                print("El barbero se echa a dormir de nuevo\n")
                Barberia.barberodormido = True
            Barberia.mutualex.release()
            
            
            
def actividadCliente(cliente, Barberia):
    time.sleep(random.randint(7,10)) # con esto pretendo que los clientes lleguen un poco más espaciados a la barbería
    print("El cliente {} ha llegado a la barbería\n".format(cliente))
    Barberia.mutualex.acquire()
    if Barberia.clientesenSala.qsize() < 3:
        Barberia.clientesenSala.put(cliente)
        Barberia.semaforosala.acquire()
        print("El cliente {} se sienta en la sala de espera\n".format(cliente))
        Barberia.mutualex.release()
    else:
        print("El cliente {} encuentra la sala de espera llena y por tanto se va\n".format(cliente))
        Barberia.mutualex.release()


if __name__=="__main__":
        barberia= Barberia()
        
        barbero = threading.Thread(target=actividadBarbero, args=(barberia,))
        clientes = []
        for i in range(15):            
           clientes.append(threading.Thread(target=actividadCliente, args=(i+1, barberia,)))
        
        barbero.start()
        for cliente in clientes:
            cliente.start()
        
        barbero.join()
        for cliente in clientes:
            cliente.join()
        
        print("El barbero ha atendido a los clientes:" +  str(barberia.clientes_atendidos))

El barbero está durmiendo...

El barbero sigue durmiendo

El barbero sigue durmiendo

El barbero sigue durmiendo

El cliente 5 ha llegado a la barbería

El cliente 5 se sienta en la sala de espera

El cliente 15 ha llegado a la barbería

El cliente 15 se sienta en la sala de espera

El cliente 6 ha llegado a la barbería

El cliente 6 se sienta en la sala de espera

El cliente 4 ha llegado a la barbería
El cliente 12 ha llegado a la barbería

El cliente 12 encuentra la sala de espera llena y por tanto se va

El cliente 10 ha llegado a la barbería

El cliente 10 encuentra la sala de espera llena y por tanto se va


El cliente 4 encuentra la sala de espera llena y por tanto se va

El cliente 2 ha llegado a la barbería

El cliente 2 encuentra la sala de espera llena y por tanto se va

El barbero se despierta 

El barbero da paso a un cliente

El barbero le está cortando el pelo al cliente 5

El cliente 1 ha llegado a la barbería

El cliente 8 ha llegado a la barbería

El cliente 9 ha llega